# ClyptQ Advanced Strategies

Advanced strategy patterns:
1. Multi-Alpha combination strategy
2. Factor Neutralization
3. Multi-Timeframe strategy
4. Risk-Adjusted Weight
5. Dynamic Universe usage

In [ ]:
%load_ext autoreload
%autoreload 2

from datetime import datetime
import pandas as pd
import numpy as np

from clyptq import operator
from clyptq.data.provider import DataProvider
from clyptq.data.spec import OHLCVSpec
from clyptq.universe import CryptoLiquid, StaticUniverse
from clyptq.strategy import Strategy
from clyptq.trading.engine import Engine

# Common settings
START = datetime(2025, 10, 15)
END = datetime(2026, 1, 5)

print("Imports successful!")

## 1. Multi-Alpha Combination Strategy

Strategy combining multiple alpha signals:
- Momentum (trend following)
- Mean Reversion
- Volume Factor (volume-based)

In [ ]:
class MultiAlphaStrategy(Strategy):
    """Multi-Alpha Combination Strategy."""
    
    name = "MultiAlpha"
    
    data = {
        "ohlcv": OHLCVSpec(
            exchange="gateio",
            market_type="spot",
            timeframe="1d",
        ),
    }
    
    universe = CryptoLiquid(top_n=30, min_dollar_volume=100_000)
    rebalance_freq = "1d"
    
    def warmup_periods(self) -> int:
        return 30
    
    def compute_signal(self):
        close = self.provider["close"]
        volume = self.provider["volume"]
        
        # Alpha 1: Momentum (20-day returns)
        returns_20d = operator.ts_returns(close, period=20)
        alpha_momentum = operator.rank(returns_20d)
        
        # Alpha 2: Mean Reversion (inverse Z-score)
        ma_20 = operator.ts_mean(close, 20)
        std_20 = operator.ts_std(close, 20)
        zscore = operator.div(
            operator.sub(close, ma_20),
            operator.add(std_20, 1e-8)
        )
        alpha_mr = operator.rank(operator.neg(zscore))
        
        # Alpha 3: Volume Surge
        vol_ma = operator.ts_mean(volume, 20)
        vol_ratio = operator.div(volume, operator.add(vol_ma, 1e-8))
        alpha_vol = operator.rank(vol_ratio)
        
        # Weighted combination
        combined = operator.ca_weighted_sum(
            alpha_momentum, alpha_mr, alpha_vol,
            weights=[0.5, 0.3, 0.2]
        )
        
        # Spot: long-only
        signal = operator.rank(combined)
        signal = operator.l1_norm(signal)
        
        return signal


# Backtest
engine = Engine()
result = engine.run(
    strategy=MultiAlphaStrategy(),
    mode="backtest",
    start=START,
    end=END,
    initial_capital=10000.0,
    market_type="spot",
    verbose=True,
)

print(f"\n=== Multi-Alpha Results ===")
if result.metrics:
    print(f"Total Return: {result.metrics.total_return:.2%}")
    print(f"Sharpe Ratio: {result.metrics.sharpe_ratio:.2f}")
    print(f"Max Drawdown: {result.metrics.max_drawdown:.2%}")

## 2. Factor Neutralization

Remove specific factor exposure to extract pure alpha

In [ ]:
class NeutralizedMomentumStrategy(Strategy):
    """Momentum neutralized by size factor."""
    
    name = "NeutralizedMomentum"
    
    data = {
        "ohlcv": OHLCVSpec(
            exchange="gateio",
            market_type="spot",
            timeframe="1d",
        ),
    }
    
    universe = CryptoLiquid(top_n=30, min_dollar_volume=100_000)
    rebalance_freq = "1d"
    
    def warmup_periods(self) -> int:
        return 30
    
    def compute_signal(self):
        close = self.provider["close"]
        volume = self.provider["volume"]
        
        # Momentum alpha
        returns_20d = operator.ts_returns(close, period=20)
        alpha = operator.rank(returns_20d)
        
        # Size factor (dollar volume as proxy)
        dollar_volume = operator.mul(close, volume)
        size_factor = operator.rank(operator.ts_mean(dollar_volume, 20))
        
        # Size neutralization: demean within size groups
        # Simplified: remove correlation with size factor
        alpha_demeaned = operator.demean(alpha)
        size_demeaned = operator.demean(size_factor)
        
        # Orthogonalization: alpha - beta * size
        # beta = cov(alpha, size) / var(size)
        cov = operator.ts_cov(alpha_demeaned, size_demeaned, window=20)
        var = operator.ts_std(size_demeaned, 20)
        var_sq = operator.mul(var, var)
        beta = operator.div(cov, operator.add(var_sq, 1e-8))
        
        # Neutralized alpha
        neutralized = operator.sub(
            alpha_demeaned,
            operator.mul(beta, size_demeaned)
        )
        
        # Spot: long-only
        signal = operator.rank(neutralized)
        signal = operator.l1_norm(signal)
        
        return signal


result = engine.run(
    strategy=NeutralizedMomentumStrategy(),
    mode="backtest",
    start=START,
    end=END,
    initial_capital=10000.0,
    market_type="spot",
    verbose=True,
)

print(f"\n=== Neutralized Momentum Results ===")
if result.metrics:
    print(f"Total Return: {result.metrics.total_return:.2%}")
    print(f"Sharpe Ratio: {result.metrics.sharpe_ratio:.2f}")
    print(f"Max Drawdown: {result.metrics.max_drawdown:.2%}")

## 3. Multi-Timeframe Strategy

Combine signals from multiple timeframes

In [ ]:
class MultiTimeframeStrategy(Strategy):
    """Multi-timeframe momentum strategy."""
    
    name = "MultiTimeframe"
    
    data = {
        "ohlcv": OHLCVSpec(
            exchange="gateio",
            market_type="spot",
            timeframe="1d",
        ),
    }
    
    universe = CryptoLiquid(top_n=30, min_dollar_volume=100_000)
    rebalance_freq = "1d"
    
    def warmup_periods(self) -> int:
        return 30
    
    def compute_signal(self):
        # Multi-timeframe data (automatically aligned)
        close_1d = self.provider["close"]
        close_3d = self.provider["close", "3d"]  # 3-day resample
        close_5d = self.provider["close", "5d"]  # 5-day resample
        
        # Momentum for each timeframe
        alpha_1d = operator.rank(operator.ts_returns(close_1d, period=5))
        alpha_3d = operator.rank(operator.ts_returns(close_3d, period=3))
        alpha_5d = operator.rank(operator.ts_returns(close_5d, period=5))
        
        # Weighted combination across timeframes
        combined = operator.ca_weighted_sum(
            alpha_1d, alpha_3d, alpha_5d,
            weights=[0.5, 0.3, 0.2]
        )
        
        # Normalize
        signal = operator.rank(combined)
        signal = operator.l1_norm(signal)
        
        return signal


result = engine.run(
    strategy=MultiTimeframeStrategy(),
    mode="backtest",
    start=START,
    end=END,
    initial_capital=10000.0,
    market_type="spot",
    verbose=True,
)

print(f"\n=== Multi-Timeframe Results ===")
if result.metrics:
    print(f"Total Return: {result.metrics.total_return:.2%}")
    print(f"Sharpe Ratio: {result.metrics.sharpe_ratio:.2f}")
    print(f"Max Drawdown: {result.metrics.max_drawdown:.2%}")

## 4. Risk-Adjusted Weight (Volatility Adjustment)

Adjust weights by inverse volatility

In [ ]:
class RiskAdjustedStrategy(Strategy):
    """Risk-adjusted momentum (inverse volatility weighted)."""
    
    name = "RiskAdjusted"
    
    data = {
        "ohlcv": OHLCVSpec(
            exchange="gateio",
            market_type="spot",
            timeframe="1d",
        ),
    }
    
    universe = CryptoLiquid(top_n=30, min_dollar_volume=100_000)
    rebalance_freq = "1d"
    
    def warmup_periods(self) -> int:
        return 30
    
    def compute_signal(self):
        close = self.provider["close"]
        
        # Basic momentum alpha
        returns = operator.ts_returns(close, period=1)
        returns_20d = operator.ts_returns(close, period=20)
        alpha = operator.rank(returns_20d)
        
        # Volatility (20-day)
        volatility = operator.ts_std(returns, 20)
        
        # Inverse volatility weight
        inv_vol = operator.div(1.0, operator.add(volatility, 1e-8))
        inv_vol_rank = operator.rank(inv_vol)
        
        # Alpha * InverseVol
        risk_adjusted = operator.mul(alpha, inv_vol_rank)
        
        # Normalize
        signal = operator.rank(risk_adjusted)
        signal = operator.l1_norm(signal)
        
        return signal


result = engine.run(
    strategy=RiskAdjustedStrategy(),
    mode="backtest",
    start=START,
    end=END,
    initial_capital=10000.0,
    market_type="spot",
    verbose=True,
)

print(f"\n=== Risk-Adjusted Results ===")
if result.metrics:
    print(f"Total Return: {result.metrics.total_return:.2%}")
    print(f"Sharpe Ratio: {result.metrics.sharpe_ratio:.2f}")
    print(f"Max Drawdown: {result.metrics.max_drawdown:.2%}")

## 5. Long/Short Futures Strategy

Long/short strategy for futures market

In [ ]:
class LongShortFuturesStrategy(Strategy):
    """Long/Short momentum strategy for futures."""
    
    name = "LongShortFutures"
    
    data = {
        "ohlcv": OHLCVSpec(
            exchange="gateio",
            market_type="spot",  # Using spot data (futures separate)
            timeframe="1d",
        ),
    }
    
    universe = CryptoLiquid(top_n=30, min_dollar_volume=100_000)
    rebalance_freq = "1d"
    
    def warmup_periods(self) -> int:
        return 30
    
    def compute_signal(self):
        close = self.provider["close"]
        
        # Momentum-based
        returns_20d = operator.ts_returns(close, period=20)
        signal = operator.rank(returns_20d)
        
        # Futures: long/short (demean)
        signal = operator.demean(signal)
        
        # Limit extreme values
        signal = operator.clip(signal, lower=-0.1, upper=0.1)
        
        # L1 normalization
        signal = operator.l1_norm(signal)
        
        return signal


result = engine.run(
    strategy=LongShortFuturesStrategy(),
    mode="backtest",
    start=START,
    end=END,
    initial_capital=10000.0,
    market_type="futures",
    leverage=2.0,
    verbose=True,
)

print(f"\n=== Long/Short Futures Results ===")
if result.metrics:
    print(f"Total Return: {result.metrics.total_return:.2%}")
    print(f"Sharpe Ratio: {result.metrics.sharpe_ratio:.2f}")
    print(f"Max Drawdown: {result.metrics.max_drawdown:.2%}")

## 6. Strategy Comparison

In [ ]:
# Run all strategies and compare
strategies = [
    ("MultiAlpha", MultiAlphaStrategy(), "spot", 1.0),
    ("Neutralized", NeutralizedMomentumStrategy(), "spot", 1.0),
    ("MultiTimeframe", MultiTimeframeStrategy(), "spot", 1.0),
    ("RiskAdjusted", RiskAdjustedStrategy(), "spot", 1.0),
    ("LongShort", LongShortFuturesStrategy(), "futures", 2.0),
]

results_summary = []

for name, strategy, market_type, leverage in strategies:
    result = engine.run(
        strategy=strategy,
        mode="backtest",
        start=START,
        end=END,
        initial_capital=10000.0,
        market_type=market_type,
        leverage=leverage,
        verbose=False,
    )
    
    if result.metrics:
        results_summary.append({
            "Strategy": name,
            "Return": f"{result.metrics.total_return:.2%}",
            "Sharpe": f"{result.metrics.sharpe_ratio:.2f}",
            "MaxDD": f"{result.metrics.max_drawdown:.2%}",
            "Trades": result.metrics.num_trades,
        })

print("=== Strategy Comparison ===")
print(pd.DataFrame(results_summary).to_string(index=False))

## Summary

### Advanced Strategy Patterns

| Pattern | Description | Key Operators |
|---------|-------------|---------------|
| Multi-Alpha | Combine multiple alphas | `ca_weighted_sum`, `ca_reduce_avg` |
| Neutralization | Remove factor exposure | `ts_cov`, `ts_std`, `demean` |
| Multi-Timeframe | Combine multiple timeframes | `provider["close", "3d"]` |
| Risk-Adjusted | Volatility adjustment | `ts_std`, `div`, `mul` |
| Long/Short | Long-short strategy | `demean`, `clip`, `l1_norm` |

### Best Practices

1. **Always use operators**: No direct pandas method calls
2. **Rank before combining**: Unify scale
3. **Spot vs Futures**: 
   - Spot: `l1_norm` (positive only)
   - Futures: `demean` → `l1_norm` (long/short)
4. **Limit extreme values**: Use `winsorize` or `clip`
5. **Proper warmup**: At least as long as longest lookback period

In [ ]:
print("Advanced Strategies test complete!")